In [1]:
import src.TX as TX
import numpy as np
import cv2
import time

#### parameters that needs to be exhanged between the sender and the receiver #####
IP = "10.42.0.15"
# IP = "0.0.0.0"
PORT = 23422
            #  t1  t2  t3  t4  t5  t6  t7  t8  t9
X = np.array([[ 1,  0,  0,  0,  0,  0,  1,  0,  0], # m1
              [ 1,  0,  0,  0,  0,  0,  0,  1,  0], # m2
              [ 1,  0,  0,  0,  0,  0,  0,  0,  1], # m3
              [ 0,  1,  0,  0,  0,  0,  1,  0,  0], # m4
              [ 0,  1,  0,  0,  0,  0,  0,  1,  0], # m5
              [ 0,  1,  0,  0,  0,  0,  0,  0,  1], # m6
              [ 0,  0,  1,  0,  0,  0,  1,  0,  0], # m7
              [ 0,  0,  1,  0,  0,  0,  0,  1,  0], # m8
              [ 0,  0,  1,  0,  0,  0,  0,  0,  1], # m9
              [ 0,  0,  0,  1,  0,  0,  1,  0,  0], # m10
              [ 0,  0,  0,  1,  0,  0,  0,  1,  0], # m11
              [ 0,  0,  0,  1,  0,  0,  0,  0,  1], # m12
              [ 0,  0,  0,  0,  1,  0,  1,  0,  0], # m13
              [ 0,  0,  0,  0,  1,  0,  0,  1,  0], # m14
              [ 0,  0,  0,  0,  1,  0,  0,  0,  1], # m15
              [ 0,  0,  0,  0,  0,  1,  1,  0,  0], # m16
              [ 0,  0,  0,  0,  0,  1,  0,  1,  0], # m17
              [ 0,  0,  0,  0,  0,  1,  0,  0,  1]]) # m18
            #  t1  t2  t3  t4  t5  t6  t7  t8  t9
Y = np.array([[ 0,  0,  0,  0,  0,  0,  1,  0,  0], # m1
              [ 0,  0,  0,  0,  0,  0,  0,  1,  0], # m2
              [ 1,  0,  0,  0,  0,  0,  0,  0,  0], # m3
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m4
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m5
              [ 0,  1,  0,  0,  0,  0,  0,  0,  0], # m6
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m7
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m8
              [ 0,  0,  1,  0,  0,  0,  0,  0,  0], # m9
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m10
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m11
              [ 0,  0,  0,  1,  0,  0,  0,  0,  0], # m12
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m13
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m14
              [ 0,  0,  0,  0,  1,  0,  0,  0,  0], # m15
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m16
              [ 0,  0,  0,  0,  0,  1,  0,  0,  0], # m17
              [ 0,  0,  0,  0,  0,  0,  0,  0,  1]]) # m18


X_eye = np.eye(18)
Y_eye = np.eye(18)
chunk_size_Byte = 1024
key = b"key"
digestmod = 'sha384'


udp_tx = TX.UDP_TX(IP=IP, PORT=PORT, X=X, Y=Y, chunk_size_Byte=chunk_size_Byte, KEY=key, digestmod=digestmod)

# with open("send.txt", "rb") as f:
#     udp_tx.transmit(f.read())

# udp_tx.transmit(b"Hello World this is not working and this is bad, but I think I need a bigger text to transmit", attack=[0])
# 1/0

cap = cv2.VideoCapture(0)  # Capture video from the default camera
# Define the compression parameters for Progressive JPEG
compression_params = [cv2.IMWRITE_JPEG_PROGRESSIVE, 1, cv2.IMWRITE_JPEG_QUALITY, 95]
## counting the number of frames per second
start_time = time.time()
frame_counter = 0
average_tx_size = 0
while True:
    ret, frame = cap.read()
    # Encode the image to JPEG format in memory
    success, encoded_frame = cv2.imencode('.jpg', frame, compression_params)
    data = encoded_frame.tobytes()
    average_tx_size+=len(data)//1000
    udp_tx.transmit(data)
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break
    time.sleep(.01)
    frame_counter += 1
    if time.time() - start_time >= 1:
        print("frame per second: ", frame_counter, "fps ,data rate: ", average_tx_size, "KB/s")
        frame_counter = 0
        start_time = time.time()
        average_tx_size = 0
cap.release()



[ WARN:0@0.364] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


frame per second:  20 fps ,data rate:  2133 KB/s
frame per second:  22 fps ,data rate:  2354 KB/s
frame per second:  27 fps ,data rate:  2887 KB/s
frame per second:  28 fps ,data rate:  2993 KB/s
frame per second:  28 fps ,data rate:  2860 KB/s
frame per second:  28 fps ,data rate:  2701 KB/s
frame per second:  27 fps ,data rate:  2912 KB/s
frame per second:  23 fps ,data rate:  2540 KB/s
frame per second:  24 fps ,data rate:  2628 KB/s
frame per second:  21 fps ,data rate:  2300 KB/s
frame per second:  19 fps ,data rate:  2089 KB/s
frame per second:  20 fps ,data rate:  2199 KB/s
frame per second:  26 fps ,data rate:  2858 KB/s
frame per second:  23 fps ,data rate:  2538 KB/s
frame per second:  22 fps ,data rate:  2423 KB/s
frame per second:  25 fps ,data rate:  2755 KB/s
frame per second:  22 fps ,data rate:  2426 KB/s
frame per second:  24 fps ,data rate:  2645 KB/s
frame per second:  23 fps ,data rate:  2537 KB/s
frame per second:  21 fps ,data rate:  2320 KB/s
frame per second:  1

In [ ]:
import time

time.time()

1725068286.1887634